In [ ]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import os
import json
import datetime
from sklearn.model_selection import KFold
from sklearn.linear_model import Ridge, Lasso, ElasticNet, BayesianRidge, HuberRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor, GradientBoostingRegressor, BaggingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_percentage_error
import xgboost as xgb
import lightgbm as lgb
import optuna
import torch

# Try to import TabPFN and TabM with fallbacks
try:
    from tabpfn import TabPFNRegressor
    TABPFN_AVAILABLE = True
except ImportError:
    print("⚠️  TabPFN not available. Install with: pip install tabpfn")
    TABPFN_AVAILABLE = False

try:
    import tabm
    if hasattr(tabm, 'TabMRegressor'):
        TabMRegressor = tabm.TabMRegressor
        TABM_AVAILABLE = True
    elif hasattr(tabm, 'TabM'):
        TabMRegressor = tabm.TabM
        TABM_AVAILABLE = True
    else:
        available_attrs = [attr for attr in dir(tabm) if not attr.startswith('_')]
        print(f"⚠️  TabM available attributes: {available_attrs}")
        TABM_AVAILABLE = False
except ImportError:
    print("⚠️  TabM not available. Install from GitHub: pip install git+https://github.com/yandex-research/tabm.git")
    TABM_AVAILABLE = False

def setup_directories():
    """Create necessary directories for submissions and logs"""
    if not os.path.exists('submissions'):
        os.makedirs('submissions')
        print("📁 Created 'submissions' directory")
    
    if not os.path.exists('logs'):
        os.makedirs('logs')
        print("📁 Created 'logs' directory")

def log_results(iteration, final_mape, individual_mapes, submission_filename, 
                level1_models, top_models, best_weights, log_file):
    """Log iteration results to file and console"""
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    log_entry = {
        "iteration": iteration,
        "timestamp": timestamp,
        "final_mape": final_mape,
        "individual_mapes": individual_mapes,
        "submission_file": submission_filename,
        "level1_models": level1_models,
        "top_models": top_models,
        "best_weights": best_weights
    }
    
    # Append to log file
    with open(log_file, 'a') as f:
        f.write(json.dumps(log_entry) + '\n')
    
    print(f"📝 Iteration {iteration} logged: MAPE = {final_mape:.6f}")

def run_single_iteration(iteration, train, test, log_file):
    """Run a single iteration of the ensemble training"""
    print(f"\n{'='*60}")
    print(f"🚀 ITERATION {iteration}/30")
    print(f"{'='*60}")
    
    # Set random seed for reproducibility with variation
    np.random.seed(42 + iteration)
    
    X = train.drop([f'BlendProperty{i}' for i in range(1, 11)], axis=1)
    y = train[[f'BlendProperty{i}' for i in range(1, 11)]]
    X_test = test.drop(['ID'], axis=1)
    
    # Use different random states for each iteration
    kf = KFold(n_splits=5, shuffle=True, random_state=42 + iteration)
    
    print("🚀 Starting Level 1 training...")
    
    # Level 1: Advanced models as base models
    level1_names = ['XGB']
    
    if TABM_AVAILABLE:
        level1_names.append('TabM')
    if TABPFN_AVAILABLE:
        level1_names.append('TabPFN')
    
    level1_names.extend(['LGBM', 'RandomForest', 'ExtraTrees'])
    
    print(f"📊 Level 1 models: {', '.join(level1_names)}")
    
    level1_oof = {name: np.zeros(y.shape) for name in level1_names}
    level1_test = {name: np.zeros((X_test.shape[0], y.shape[1])) for name in level1_names}
    
    for t in range(y.shape[1]):
        print(f"🎯 Training Level 1 for target BlendProperty{t+1}...")
        for fold, (tr_idx, val_idx) in enumerate(kf.split(X)):
            X_tr, X_val = X.iloc[tr_idx], X.iloc[val_idx]
            y_tr, y_val = y.iloc[tr_idx, t], y.iloc[val_idx, t]
            
            # XGBoost with some randomization
            xgb_model = xgb.XGBRegressor(
                n_estimators=np.random.randint(150, 250),
                max_depth=np.random.randint(5, 8),
                learning_rate=np.random.uniform(0.05, 0.15),
                subsample=np.random.uniform(0.7, 0.9),
                colsample_bytree=np.random.uniform(0.7, 0.9),
                verbosity=0,
                n_jobs=-1,
                random_state=42 + iteration
            )
            xgb_model.fit(X_tr, y_tr)
            level1_oof['XGB'][val_idx, t] = xgb_model.predict(X_val)
            level1_test['XGB'][:, t] += xgb_model.predict(X_test) / kf.n_splits
            
            # LightGBM with randomization
            lgb_model = lgb.LGBMRegressor(
                n_estimators=np.random.randint(150, 250),
                max_depth=np.random.randint(5, 8),
                learning_rate=np.random.uniform(0.05, 0.15),
                subsample=np.random.uniform(0.7, 0.9),
                colsample_bytree=np.random.uniform(0.7, 0.9),
                verbose=-1,
                n_jobs=-1,
                random_state=42 + iteration
            )
            lgb_model.fit(X_tr, y_tr)
            level1_oof['LGBM'][val_idx, t] = lgb_model.predict(X_val)
            level1_test['LGBM'][:, t] += lgb_model.predict(X_test) / kf.n_splits
            
            # RandomForest with randomization
            rf = RandomForestRegressor(
                n_estimators=np.random.randint(150, 250),
                max_depth=np.random.randint(8, 12),
                n_jobs=-1,
                random_state=42 + iteration
            )
            rf.fit(X_tr, y_tr)
            level1_oof['RandomForest'][val_idx, t] = rf.predict(X_val)
            level1_test['RandomForest'][:, t] += rf.predict(X_test) / kf.n_splits
            
            # ExtraTrees with randomization
            et = ExtraTreesRegressor(
                n_estimators=np.random.randint(150, 250),
                max_depth=np.random.randint(8, 12),
                n_jobs=-1,
                random_state=42 + iteration
            )
            et.fit(X_tr, y_tr)
            level1_oof['ExtraTrees'][val_idx, t] = et.predict(X_val)
            level1_test['ExtraTrees'][:, t] += et.predict(X_test) / kf.n_splits
            
            # TabM (if available)
            if 'TabM' in level1_names and TABM_AVAILABLE:
                try:
                    tabm_model = TabMRegressor(
                        device='cuda' if torch.cuda.is_available() else 'cpu'
                    )
                    tabm_model.fit(X_tr.values, y_tr.values)
                    level1_oof['TabM'][val_idx, t] = tabm_model.predict(X_val.values)
                    level1_test['TabM'][:, t] += tabm_model.predict(X_test.values) / kf.n_splits
                except Exception as e:
                    print(f"    Error with TabM: {e}")
                    if 'TabM' in level1_names:
                        level1_names.remove('TabM')
            
            # TabPFN (if available)
            if 'TabPFN' in level1_names and TABPFN_AVAILABLE:
                try:
                    tabpfn = TabPFNRegressor(device='cuda' if torch.cuda.is_available() else 'cpu')
                    tabpfn.fit(X_tr, y_tr.values)
                    level1_oof['TabPFN'][val_idx, t] = tabpfn.predict(X_val)
                    level1_test['TabPFN'][:, t] += tabpfn.predict(X_test) / kf.n_splits
                except Exception as e:
                    print(f"    Error with TabPFN: {e}")
                    if 'TabPFN' in level1_names:
                        level1_names.remove('TabPFN')
    
    # Remove empty entries
    level1_names = [name for name in level1_names if name in level1_oof and level1_oof[name].sum() != 0]
    level1_oof = {name: level1_oof[name] for name in level1_names}
    level1_test = {name: level1_test[name] for name in level1_names}
    
    print(f"\n📊 Level 1 MAPE Scores (using {len(level1_names)} models):")
    for name in level1_names:
        mape = mean_absolute_percentage_error(y, level1_oof[name])
        print(f"  {name}: {mape:.6f}")
    
    # Prepare Level 2 inputs
    stack_X = np.concatenate([level1_oof[name] for name in level1_names], axis=1)
    stack_X_test = np.concatenate([level1_test[name] for name in level1_names], axis=1)
    
    print("\n🚀 Starting Level 2 stacking...")
    
    # Level 2 models
    level2_names = [
        'Ridge', 'Lasso', 'ElasticNet', 'BayesianRidge', 'Huber',
        'RandomForest_L2', 'ExtraTrees_L2', 'AdaBoost', 'GradientBoost', 'Bagging',
        'KNN', 'SVR', 'XGB_L2', 'LGBM_L2'
    ]
    level2_oof = {name: np.zeros(y.shape) for name in level2_names}
    level2_test = {name: np.zeros((X_test.shape[0], y.shape[1])) for name in level2_names}
    
    for t in range(y.shape[1]):
        for fold, (tr_idx, val_idx) in enumerate(kf.split(stack_X)):
            X_tr, X_val = stack_X[tr_idx], stack_X[val_idx]
            y_tr, y_val = y.iloc[tr_idx, t], y.iloc[val_idx, t]
            
            # Train all Level 2 models (keeping the same structure but with some randomization)
            models = {
                'Ridge': Ridge(alpha=np.random.uniform(0.5, 2.0)),
                'Lasso': Lasso(alpha=np.random.uniform(0.05, 0.2), max_iter=1000),
                'ElasticNet': ElasticNet(alpha=np.random.uniform(0.05, 0.2), max_iter=1000),
                'BayesianRidge': BayesianRidge(),
                'Huber': HuberRegressor(),
                'RandomForest_L2': RandomForestRegressor(n_estimators=50, random_state=42 + iteration),
                'ExtraTrees_L2': ExtraTreesRegressor(n_estimators=50, random_state=42 + iteration),
                'AdaBoost': AdaBoostRegressor(n_estimators=50, random_state=42 + iteration),
                'GradientBoost': GradientBoostingRegressor(n_estimators=100, random_state=42 + iteration),
                'Bagging': BaggingRegressor(n_estimators=50, random_state=42 + iteration),
                'KNN': KNeighborsRegressor(n_neighbors=np.random.randint(3, 8)),
                'SVR': SVR(kernel='rbf', gamma='scale'),
                'XGB_L2': xgb.XGBRegressor(verbosity=0, random_state=42 + iteration),
                'LGBM_L2': lgb.LGBMRegressor(verbose=-1, random_state=42 + iteration)
            }
            
            for name, model in models.items():
                model.fit(X_tr, y_tr)
                level2_oof[name][val_idx, t] = model.predict(X_val)
                level2_test[name][:, t] += model.predict(stack_X_test) / kf.n_splits
    
    print("\n📊 Level 2 MAPE Scores:")
    level2_scores = {}
    for name in level2_names:
        mape = mean_absolute_percentage_error(y, level2_oof[name])
        level2_scores[name] = mape
        print(f"  {name}: {mape:.6f}")
    
    # Dynamic selection of top performers
    TOP_K = min(6, len(level2_names))
    sorted_models = sorted(level2_scores.items(), key=lambda x: x[1])
    top_models = [model[0] for model in sorted_models[:TOP_K]]
    
    print(f"\n🎯 Selected top {len(top_models)} models for Level 3:")
    for i, model in enumerate(top_models):
        print(f"  {i+1}. {model}: {level2_scores[model]:.6f}")
    
    # Level 3 optimization
    def objective(trial):
        weights = {}
        for model in top_models:
            weights[model] = trial.suggest_float(f'w_{model}', 0.0, 1.0)
        
        total_weight = sum(weights.values())
        if total_weight == 0:
            return float('inf')
        
        normalized_weights = {k: v/total_weight for k, v in weights.items()}
        ensemble_pred = sum(normalized_weights[model] * level2_oof[model] 
                           for model in top_models)
        
        mape = mean_absolute_percentage_error(y, ensemble_pred)
        return mape
    
    print("🔧 Optimizing ensemble weights...")
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=200, show_progress_bar=False)
    
    # Get best weights
    best_params = study.best_params
    total_weight = sum(best_params.values())
    normalized_weights = {k: v/total_weight for k, v in best_params.items()}
    
    # Create final predictions
    final_test = sum(normalized_weights[f'w_{model}'] * level2_test[model] 
                     for model in top_models)
    
    final_oof = sum(normalized_weights[f'w_{model}'] * level2_oof[model] 
                    for model in top_models)
    
    final_mape = mean_absolute_percentage_error(y, final_oof)
    
    # Calculate individual target MAPE scores
    individual_mapes = {}
    for i in range(y.shape[1]):
        target_mape = mean_absolute_percentage_error(y.iloc[:, i], final_oof[:, i])
        individual_mapes[f'BlendProperty{i+1}'] = target_mape
    
    # Create submission
    submission = pd.DataFrame(final_test, columns=[f'BlendProperty{i}' for i in range(1, 11)])
    submission.insert(0, 'ID', test['ID'])
    
    # Save with descriptive filename
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    submission_filename = f"submission_iter{iteration:02d}_{timestamp}_mape{final_mape:.6f}.csv"
    submission_filepath = os.path.join('submissions', submission_filename)
    submission.to_csv(submission_filepath, index=False)
    
    # Log results
    log_results(iteration, final_mape, individual_mapes, submission_filename,
                level1_names, top_models, normalized_weights, log_file)
    
    print(f"\n🎉 Iteration {iteration} completed:")
    print(f"  Final MAPE: {final_mape:.6f}")
    print(f"  Submission saved: {submission_filename}")
    
    return final_mape, submission_filename

def main():
    """Main function to run 30 iterations"""
    print("🚀 Starting Automated Ensemble Training - 30 Iterations")
    print("="*70)
    
    # Setup directories
    setup_directories()
    
    # Load data
    print("Loading data...")
    train = pd.read_csv('train.csv')
    test = pd.read_csv('test.csv')
    
    # Setup log file
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    log_file = os.path.join('logs', f'ensemble_log_{timestamp}.jsonl')
    
    # Initialize summary tracking
    all_results = []
    
    print(f"📝 Log file: {log_file}")
    print("="*70)
    
    # Run 30 iterations
    for iteration in range(1, 31):
        try:
            mape, filename = run_single_iteration(iteration, train, test, log_file)
            all_results.append((iteration, mape, filename))
            
            # Print progress
            print(f"\n📊 Progress: {iteration}/30 iterations completed")
            if len(all_results) > 1:
                best_mape = min(all_results, key=lambda x: x[1])
                print(f"🏆 Best so far: Iteration {best_mape[0]} with MAPE {best_mape[1]:.6f}")
            
        except Exception as e:
            print(f"❌ Error in iteration {iteration}: {str(e)}")
            continue
    
    # Final summary
    print("\n" + "="*70)
    print("🎉 ALL ITERATIONS COMPLETED!")
    print("="*70)
    
    if all_results:
        # Sort by MAPE
        all_results.sort(key=lambda x: x[1])
        
        print(f"\n📊 FINAL SUMMARY ({len(all_results)} successful runs):")
        print(f"🏆 Best Model: Iteration {all_results[0][0]} - MAPE: {all_results[0][1]:.6f}")
        print(f"   File: {all_results[0][2]}")
        
        print(f"\n📈 Top 5 Results:")
        for i, (iteration, mape, filename) in enumerate(all_results[:5]):
            print(f"  {i+1}. Iteration {iteration:2d}: {mape:.6f} - {filename}")
        
        print(f"\n📉 Performance Statistics:")
        mapes = [result[1] for result in all_results]
        print(f"  Average MAPE: {np.mean(mapes):.6f}")
        print(f"  Std Dev:      {np.std(mapes):.6f}")
        print(f"  Min MAPE:     {np.min(mapes):.6f}")
        print(f"  Max MAPE:     {np.max(mapes):.6f}")
        
        # Create summary CSV
        summary_df = pd.DataFrame(all_results, columns=['Iteration', 'MAPE', 'Filename'])
        summary_file = os.path.join('logs', f'summary_{timestamp}.csv')
        summary_df.to_csv(summary_file, index=False)
        print(f"\n💾 Summary saved to: {summary_file}")
    
    print(f"\n📁 All files saved in:")
    print(f"  Submissions: ./submissions/")
    print(f"  Logs: ./logs/")
    
    print("\n✅ Automated training completed successfully!")

if __name__ == "__main__":
    main()

🚀 Starting Automated Ensemble Training - 30 Iterations
📁 Created 'logs' directory
Loading data...
📝 Log file: logs\ensemble_log_20250723_093848.jsonl

🚀 ITERATION 1/30
🚀 Starting Level 1 training...
📊 Level 1 models: XGB, TabM, TabPFN, LGBM, RandomForest, ExtraTrees
🎯 Training Level 1 for target BlendProperty1...
    Error with TabM: TabM.__init__() missing 1 required keyword-only argument: 'd_out'
🎯 Training Level 1 for target BlendProperty2...
🎯 Training Level 1 for target BlendProperty3...
🎯 Training Level 1 for target BlendProperty4...
🎯 Training Level 1 for target BlendProperty5...
🎯 Training Level 1 for target BlendProperty6...
🎯 Training Level 1 for target BlendProperty7...
🎯 Training Level 1 for target BlendProperty8...
🎯 Training Level 1 for target BlendProperty9...
🎯 Training Level 1 for target BlendProperty10...

📊 Level 1 MAPE Scores (using 5 models):
  XGB: 1.370306
  TabPFN: 0.545743
  LGBM: 1.215147
  RandomForest: 2.299264
  ExtraTrees: 2.294376

🚀 Starting Level 2 stac

[I 2025-07-23 09:57:19,519] A new study created in memory with name: no-name-65cfd436-6327-4117-8d58-2029704ffc00
[I 2025-07-23 09:57:19,527] Trial 0 finished with value: 0.4667318304400593 and parameters: {'w_Bagging': 0.17694027623156117, 'w_RandomForest_L2': 0.03817612808297344, 'w_ExtraTrees_L2': 0.5827377601741549, 'w_Huber': 0.5834652365884243, 'w_Ridge': 0.06818122588685072, 'w_BayesianRidge': 0.23070613360349512}. Best is trial 0 with value: 0.4667318304400593.
[I 2025-07-23 09:57:19,531] Trial 1 finished with value: 0.4621990678329909 and parameters: {'w_Bagging': 0.9391284158335836, 'w_RandomForest_L2': 0.11366903524290228, 'w_ExtraTrees_L2': 0.4878281463411367, 'w_Huber': 0.3364059992475329, 'w_Ridge': 0.5042974794562818, 'w_BayesianRidge': 0.20038608774011324}. Best is trial 1 with value: 0.4621990678329909.
[I 2025-07-23 09:57:19,535] Trial 2 finished with value: 0.4671590396029274 and parameters: {'w_Bagging': 0.5594628445221954, 'w_RandomForest_L2': 0.3816972017995788, '


📊 Level 2 MAPE Scores:
  Ridge: 0.490991
  Lasso: 0.550788
  ElasticNet: 0.658701
  BayesianRidge: 0.494784
  Huber: 0.488975
  RandomForest_L2: 0.468376
  ExtraTrees_L2: 0.480523
  AdaBoost: 0.853180
  GradientBoost: 0.529343
  Bagging: 0.467190
  KNN: 2.139914
  SVR: 0.690571
  XGB_L2: 0.618674
  LGBM_L2: 0.602255

🎯 Selected top 6 models for Level 3:
  1. Bagging: 0.467190
  2. RandomForest_L2: 0.468376
  3. ExtraTrees_L2: 0.480523
  4. Huber: 0.488975
  5. Ridge: 0.490991
  6. BayesianRidge: 0.494784
🔧 Optimizing ensemble weights...


[I 2025-07-23 09:57:19,707] Trial 14 finished with value: 0.4612280318728626 and parameters: {'w_Bagging': 0.9982114445272072, 'w_RandomForest_L2': 0.9741849749628139, 'w_ExtraTrees_L2': 0.9533132283055202, 'w_Huber': 0.9537366125497128, 'w_Ridge': 0.011875688764190206, 'w_BayesianRidge': 0.707134964907708}. Best is trial 4 with value: 0.46055036503375585.
[I 2025-07-23 09:57:19,731] Trial 15 finished with value: 0.46198309948359534 and parameters: {'w_Bagging': 0.6582001684131034, 'w_RandomForest_L2': 0.22226109437505978, 'w_ExtraTrees_L2': 0.4964617839436527, 'w_Huber': 0.595736313766696, 'w_Ridge': 0.41669459141327997, 'w_BayesianRidge': 0.12785221102139113}. Best is trial 4 with value: 0.46055036503375585.
[I 2025-07-23 09:57:19,757] Trial 16 finished with value: 0.4657081585046162 and parameters: {'w_Bagging': 0.3614635022578857, 'w_RandomForest_L2': 0.4859820247779908, 'w_ExtraTrees_L2': 0.7785408554862486, 'w_Huber': 0.5274350660265124, 'w_Ridge': 0.6479643598166591, 'w_Bayesian

❌ Error in iteration 1: Cannot save file into a non-existent directory: 'submissions'

🚀 ITERATION 2/30
🚀 Starting Level 1 training...
📊 Level 1 models: XGB, TabM, TabPFN, LGBM, RandomForest, ExtraTrees
🎯 Training Level 1 for target BlendProperty1...
    Error with TabM: TabM.__init__() missing 1 required keyword-only argument: 'd_out'
🎯 Training Level 1 for target BlendProperty2...
🎯 Training Level 1 for target BlendProperty3...
🎯 Training Level 1 for target BlendProperty4...
🎯 Training Level 1 for target BlendProperty5...
🎯 Training Level 1 for target BlendProperty6...
🎯 Training Level 1 for target BlendProperty7...
🎯 Training Level 1 for target BlendProperty8...
🎯 Training Level 1 for target BlendProperty9...
🎯 Training Level 1 for target BlendProperty10...

📊 Level 1 MAPE Scores (using 5 models):
  XGB: 1.426463
  TabPFN: 0.468050
  LGBM: 1.296142
  RandomForest: 2.518017
  ExtraTrees: 2.431582

🚀 Starting Level 2 stacking...


...

📝 Iteration 23 logged: MAPE = 0.499808

🎉 Iteration 23 completed:
  Final MAPE: 0.499808
  Submission saved: submission_iter23_20250723_155108_mape0.499808.csv

📊 Progress: 23/30 iterations completed
🏆 Best so far: Iteration 21 with MAPE 0.442845

🚀 ITERATION 24/30
🚀 Starting Level 1 training...
📊 Level 1 models: XGB, TabM, TabPFN, LGBM, RandomForest, ExtraTrees
🎯 Training Level 1 for target BlendProperty1...
    Error with TabM: TabM.__init__() missing 1 required keyword-only argument: 'd_out'
🎯 Training Level 1 for target BlendProperty2...
🎯 Training Level 1 for target BlendProperty3...
🎯 Training Level 1 for target BlendProperty4...
🎯 Training Level 1 for target BlendProperty5...
🎯 Training Level 1 for target BlendProperty6...
🎯 Training Level 1 for target BlendProperty7...
🎯 Training Level 1 for target BlendProperty8...
🎯 Training Level 1 for target BlendProperty9...
🎯 Training Level 1 for target BlendProperty10...

📊 Level 1 MAPE Scores (using 5 models):
  XGB: 1.187859
  TabPFN: 

[I 2025-07-23 16:06:34,926] A new study created in memory with name: no-name-1ed14ae1-62b5-46fe-9b11-6aea8689acdf
[I 2025-07-23 16:06:34,930] Trial 0 finished with value: 0.5193445267674933 and parameters: {'w_Huber': 0.10989110820452264, 'w_Lasso': 0.8333414962863913, 'w_Ridge': 0.2418787920091573, 'w_BayesianRidge': 0.2173385765134216, 'w_XGB_L2': 0.17020924701824147, 'w_GradientBoost': 0.36499671696225744}. Best is trial 0 with value: 0.5193445267674933.
[I 2025-07-23 16:06:34,934] Trial 1 finished with value: 0.5283554206939842 and parameters: {'w_Huber': 0.792203430091137, 'w_Lasso': 0.09384753420398495, 'w_Ridge': 0.9336602038887108, 'w_BayesianRidge': 0.44746145621271005, 'w_XGB_L2': 0.5275108151853546, 'w_GradientBoost': 0.8715283567379943}. Best is trial 0 with value: 0.5193445267674933.
[I 2025-07-23 16:06:34,936] Trial 2 finished with value: 0.532841598566238 and parameters: {'w_Huber': 0.2986772689440824, 'w_Lasso': 0.19396524039355167, 'w_Ridge': 0.402117145032834, 'w_Baye


📊 Level 2 MAPE Scores:
  Ridge: 0.544325
  Lasso: 0.531980
  ElasticNet: 0.714751
  BayesianRidge: 0.545660
  Huber: 0.531026
  RandomForest_L2: 0.599632
  ExtraTrees_L2: 0.581415
  AdaBoost: 0.837226
  GradientBoost: 0.580921
  Bagging: 0.594050
  KNN: 2.143997
  SVR: 0.702563
  XGB_L2: 0.555723
  LGBM_L2: 0.584370

🎯 Selected top 6 models for Level 3:
  1. Huber: 0.531026
  2. Lasso: 0.531980
  3. Ridge: 0.544325
  4. BayesianRidge: 0.545660
  5. XGB_L2: 0.555723
  6. GradientBoost: 0.580921
🔧 Optimizing ensemble weights...


[I 2025-07-23 16:06:35,110] Trial 16 finished with value: 0.5169068309966665 and parameters: {'w_Huber': 0.45198527740135697, 'w_Lasso': 0.7758215591639934, 'w_Ridge': 0.5521776188100315, 'w_BayesianRidge': 0.5472137051925896, 'w_XGB_L2': 0.557417553045336, 'w_GradientBoost': 0.3193561908560265}. Best is trial 4 with value: 0.5155016324714755.
[I 2025-07-23 16:06:35,131] Trial 17 finished with value: 0.5150094300227352 and parameters: {'w_Huber': 0.15398470232597272, 'w_Lasso': 0.4915001076231726, 'w_Ridge': 0.1266567076543138, 'w_BayesianRidge': 0.32608030565488955, 'w_XGB_L2': 0.2827171723306977, 'w_GradientBoost': 0.10285695811060924}. Best is trial 17 with value: 0.5150094300227352.
[I 2025-07-23 16:06:35,154] Trial 18 finished with value: 0.5200518286704308 and parameters: {'w_Huber': 0.16902341692920847, 'w_Lasso': 0.446666790623946, 'w_Ridge': 0.6779947166603746, 'w_BayesianRidge': 0.3414811493513107, 'w_XGB_L2': 0.7533431931129583, 'w_GradientBoost': 0.5118516884804728}. Best i

📝 Iteration 24 logged: MAPE = 0.511931

🎉 Iteration 24 completed:
  Final MAPE: 0.511931
  Submission saved: submission_iter24_20250723_160639_mape0.511931.csv

📊 Progress: 24/30 iterations completed
🏆 Best so far: Iteration 21 with MAPE 0.442845

🚀 ITERATION 25/30
🚀 Starting Level 1 training...
📊 Level 1 models: XGB, TabM, TabPFN, LGBM, RandomForest, ExtraTrees
🎯 Training Level 1 for target BlendProperty1...
    Error with TabM: TabM.__init__() missing 1 required keyword-only argument: 'd_out'
🎯 Training Level 1 for target BlendProperty2...
🎯 Training Level 1 for target BlendProperty3...
🎯 Training Level 1 for target BlendProperty4...
🎯 Training Level 1 for target BlendProperty5...
🎯 Training Level 1 for target BlendProperty6...
🎯 Training Level 1 for target BlendProperty7...
🎯 Training Level 1 for target BlendProperty8...
🎯 Training Level 1 for target BlendProperty9...
🎯 Training Level 1 for target BlendProperty10...

📊 Level 1 MAPE Scores (using 5 models):
  XGB: 1.514278
  TabPFN: 